# NHL Stats Preparation
## Collecting NHL Players' Stats from EliteProspects
This step gathers player performance data across leagues and seasons for downstream NHL prediction.

- Collect league-wide player stats for a given season from
https://eliteprospects.com/league/{league}/stats/{season}?page={n}

- Extract metadata, including individual player profile links.

- Build a unique player list from seasons 2000–2025 by storing profile links in a set.

- Scrape each player’s page to retrieve their first 5 years of NHL stats, filtered by seasons where they played more than 30 games.

- Gather pre-NHL stats for each player (junior, college, international, etc.).

### Import Libraries

In [1]:
import eliteprospects_scraper_api as ep
import pandas as pd
import os
import time
import random

### Collecting NHL Players' Stats from EliteProspects


In [2]:
if not os.path.exists('./data/nhl/nhl_players_metadata.csv'):
    # Collect players from 2000-2025 season
    for i in range(0, 25):
        season = f'20{str(i).zfill(2)}-20{str(i + 1).zfill(2)}'
        print(f'Scraping {season}')
        nhl_players = ep.get_season_roster("nhl", season)
        nhl_players.to_csv(f'./data/nhl/players/nhl_players_{season}.csv', index=False, encoding='utf-8-sig')
        print(f'Finished scraping {season}')
else:
    print('NHL players metadata already exists. Skipping scraping.')

NHL players metadata already exists. Skipping scraping.


In [3]:
if not os.path.exists('./data/nhl/nhl_players_metadata.csv'):
    # Collect players' metadata from 2000-2025 season
    nhl_players_metadata = pd.DataFrame()
    for i in range(0, 25):
        season = f'20{str(i).zfill(2)}-20{str(i + 1).zfill(2)}'
        nhl_players = pd.read_csv(f'./data/nhl/players/nhl_players_{season}.csv')
        nhl_players_metadata = pd.concat([nhl_players_metadata, ep.get_players_metadata(nhl_players)]).reset_index(drop=True)
        # Remove duplicates
        nhl_players_metadata = nhl_players_metadata.drop_duplicates(subset=['playername']).reset_index(drop=True)
        print(f'Finished Merging Metadata for {season}')
else:
    print('NHL players metadata already exists. Skipping merging.')
    nhl_players_metadata = pd.read_csv('./data/nhl/nhl_players_metadata.csv')


NHL players metadata already exists. Skipping merging.


### Define Function to Get Player Stats

In [4]:
# Define output file
output_path = './data/nhl/stats/nhl_players_stats.csv'

In [5]:
def get_players_stats_by_batch(players_to_scrape):
    curr_len = len(players_to_scrape)
    fail_count = 0

    for i in range(curr_len):
        player_metadata = players_to_scrape.iloc[i]
        player_name = player_metadata['playername']
        player_url = player_metadata['link']
        print(f"\n [{i + 1}] Collecting stats for {player_name} at {player_url}")

        try:
            player_stats = ep.get_player_stats(player_metadata)

            # Write to CSV file
            if os.path.exists(output_path):
                player_stats.to_csv(output_path, mode='a', header=False, index=False, encoding='utf-8-sig')
            else:
                player_stats.to_csv(output_path, index=False, encoding='utf-8-sig')
            print(f'Successfully scraped stats for {player_name}')
            
            # Print Fail Rate
            print(f'Failed rate: {fail_count / (i + 1):.2f}')

            # Add random sleep to prevent getting blocked
            if i < curr_len - 1:
                sleep_time = random.uniform(10, 120)
                print(f"Sleep for {sleep_time / 60:.2f} minutes to prevent getting blocked")
                time.sleep(sleep_time) 
        except Exception as e:
            print(f"Failed to get stats for {player_name}: {e}")

            fail_count += 1

            if i < curr_len - 1:
                # Sleep for 15-60 seconds before trying the next player
                sleep_time = random.uniform(15, 60)
                print(f"Sleeping for {sleep_time / 60:.2f} seconds before trying the next player")
                time.sleep(sleep_time)

### Fetch Players' Stats by Batch

In [6]:
# Scrape players from 900-950
curr_batch_metadata = nhl_players_metadata[900:950]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Jiri Dopita at https://www.eliteprospects.com/player/10873/jiri-dopita
Successfully scraped stats for Jiri Dopita
Failed rate: 0.00
Sleep for 0.46 minutes to prevent getting blocked

 [2] Collecting stats for Zdeno Cíger at https://www.eliteprospects.com/player/69982/zdeno-ciger
Successfully scraped stats for Zdeno Cíger
Failed rate: 0.00
Sleep for 0.89 minutes to prevent getting blocked

 [3] Collecting stats for Richard Park at https://www.eliteprospects.com/player/5214/richard-park
Error scraping Richard Park: Message: 

Failed to get stats for Richard Park: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.48 seconds before trying the next player

 [4] Collecting stats for Andreas Johansson at https://www.eliteprospects.com/player/696/andreas-johansson
Successfully scraped stats for Andreas Johansson
Failed rate: 0.25
Sleep for 1.60 minutes to prevent getting blocked

 [5] Collecting stats for Bryan Berard at https://www.eliteprospects.com/player

In [7]:
# Scrape players from 950-1000
curr_batch_metadata = nhl_players_metadata[950:1000]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Kelly Fairchild at https://www.eliteprospects.com/player/15424/kelly-fairchild
Successfully scraped stats for Kelly Fairchild
Failed rate: 0.00
Sleep for 1.57 minutes to prevent getting blocked

 [2] Collecting stats for Stephen Weiss at https://www.eliteprospects.com/player/8817/stephen-weiss
Successfully scraped stats for Stephen Weiss
Failed rate: 0.00
Sleep for 1.04 minutes to prevent getting blocked

 [3] Collecting stats for Craig MacDonald at https://www.eliteprospects.com/player/8952/craig-macdonald
Successfully scraped stats for Craig MacDonald
Failed rate: 0.00
Sleep for 0.83 minutes to prevent getting blocked

 [4] Collecting stats for Andrej Nedorost at https://www.eliteprospects.com/player/9672/andrej-nedorost
Ad or overlay is blocking dropdown for Andrej Nedorost. Trying to remove it...
Failed to remove overlay/ad.
Failed to get 'Regular Season + Postseason' table: Message: element click intercepted: Element <div class=" css-x1uf2d-control">...<

In [8]:
# Scrape players from 1000-1050
curr_batch_metadata = nhl_players_metadata[1000:1050]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Henrik Tallinder at https://www.eliteprospects.com/player/1955/henrik-tallinder
Ad or overlay is blocking dropdown for Henrik Tallinder. Trying to remove it...
Failed to remove overlay/ad.
Failed to get 'Regular Season + Postseason' table: Message: element click intercepted: Element <div class=" css-x1uf2d-control">...</div> is not clickable at point (417, 19). Other element would receive the click: <aside class="AdSlot_centering__vHSRy AdSlot_panorama__Eaw_j LayoutAds_slotH__O7qAf">...</aside>
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
	GetHandleVerifier [0x0x5f1a33+62339]
	GetHandleVerifier [0x0x5f1a74+62404]
	(No symbol) [0x0x432123]
	(No symbol) [0x0x480bb0]
	(No symbol) [0x0x47ef6a]
	(No symbol) [0x0x47cac7]
	(No symbol) [0x0x47bd7d]
	(No symbol) [0x0x470515]
	(No symbol) [0x0x49f3bc]
	(No symbo

In [9]:
# Scrape players from 1050-1100
curr_batch_metadata = nhl_players_metadata[1050:1100]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Lasse Pirjetä at https://www.eliteprospects.com/player/1798/lasse-pirjeta
Successfully scraped stats for Lasse Pirjetä
Failed rate: 0.00
Sleep for 1.25 minutes to prevent getting blocked

 [2] Collecting stats for Jason Spezza at https://www.eliteprospects.com/player/3661/jason-spezza
Successfully scraped stats for Jason Spezza
Failed rate: 0.00
Sleep for 1.43 minutes to prevent getting blocked

 [3] Collecting stats for Pierre-Marc Bouchard at https://www.eliteprospects.com/player/9086/pierre-marc-bouchard
Successfully scraped stats for Pierre-Marc Bouchard
Failed rate: 0.00
Sleep for 1.30 minutes to prevent getting blocked

 [4] Collecting stats for Jim Fahey at https://www.eliteprospects.com/player/9150/jim-fahey
Successfully scraped stats for Jim Fahey
Failed rate: 0.00
Sleep for 1.08 minutes to prevent getting blocked

 [5] Collecting stats for Jamie Lundmark at https://www.eliteprospects.com/player/8759/jamie-lundmark
Successfully scraped stats for Jami

In [10]:
# Scrape players from 1100-1150
curr_batch_metadata = nhl_players_metadata[1100:1150]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Shawn Thornton at https://www.eliteprospects.com/player/11339/shawn-thornton
Successfully scraped stats for Shawn Thornton
Failed rate: 0.00
Sleep for 1.03 minutes to prevent getting blocked

 [2] Collecting stats for Jason King at https://www.eliteprospects.com/player/9920/jason-king
Successfully scraped stats for Jason King
Failed rate: 0.00
Sleep for 0.87 minutes to prevent getting blocked

 [3] Collecting stats for Chris Schmidt at https://www.eliteprospects.com/player/13963/chris-schmidt
Successfully scraped stats for Chris Schmidt
Failed rate: 0.00
Sleep for 1.21 minutes to prevent getting blocked

 [4] Collecting stats for Garnet Exelby at https://www.eliteprospects.com/player/8558/garnet-exelby
Successfully scraped stats for Garnet Exelby
Failed rate: 0.00
Sleep for 0.87 minutes to prevent getting blocked

 [5] Collecting stats for Burke Henry at https://www.eliteprospects.com/player/9750/burke-henry
Successfully scraped stats for Burke Henry
Failed r

In [11]:
# Scrape players from 1150-1200
curr_batch_metadata = nhl_players_metadata[1150:1200]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Kent McDonell at https://www.eliteprospects.com/player/8745/kent-mcdonell
Successfully scraped stats for Kent McDonell
Failed rate: 0.00
Sleep for 0.47 minutes to prevent getting blocked

 [2] Collecting stats for Patrick Sharp at https://www.eliteprospects.com/player/8880/patrick-sharp
Successfully scraped stats for Patrick Sharp
Failed rate: 0.00
Sleep for 1.57 minutes to prevent getting blocked

 [3] Collecting stats for Alain Nasreddine at https://www.eliteprospects.com/player/11042/alain-nasreddine
Successfully scraped stats for Alain Nasreddine
Failed rate: 0.00
Sleep for 1.54 minutes to prevent getting blocked

 [4] Collecting stats for Ross Lupaschuk at https://www.eliteprospects.com/player/5979/ross-lupaschuk
Successfully scraped stats for Ross Lupaschuk
Failed rate: 0.00
Sleep for 0.62 minutes to prevent getting blocked

 [5] Collecting stats for Zdenek Blatny at https://www.eliteprospects.com/player/8636/zdenek-blatny
Successfully scraped stats for

In [12]:
# Scrape players from 1200-1250
curr_batch_metadata = nhl_players_metadata[1200:1250]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Fyodor Tyutin at https://www.eliteprospects.com/player/8691/fyodor-tyutin
Successfully scraped stats for Fyodor Tyutin
Failed rate: 0.00
Sleep for 1.89 minutes to prevent getting blocked

 [2] Collecting stats for Tim Gleason at https://www.eliteprospects.com/player/9162/tim-gleason
Successfully scraped stats for Tim Gleason
Failed rate: 0.00
Sleep for 0.92 minutes to prevent getting blocked

 [3] Collecting stats for Travis Moen at https://www.eliteprospects.com/player/8535/travis-moen
Successfully scraped stats for Travis Moen
Failed rate: 0.00
Sleep for 1.48 minutes to prevent getting blocked

 [4] Collecting stats for Jed Ortmeyer at https://www.eliteprospects.com/player/8942/jed-ortmeyer
Successfully scraped stats for Jed Ortmeyer
Failed rate: 0.00
Sleep for 1.86 minutes to prevent getting blocked

 [5] Collecting stats for Trevor Daley at https://www.eliteprospects.com/player/9128/trevor-daley
Successfully scraped stats for Trevor Daley
Failed rate: 0.0

In [13]:
# Scrape players from 1250-1300
curr_batch_metadata = nhl_players_metadata[1250:1300]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Dan Fritsche at https://www.eliteprospects.com/player/9434/dan-fritsche
Successfully scraped stats for Dan Fritsche
Failed rate: 0.00
Sleep for 0.31 minutes to prevent getting blocked

 [2] Collecting stats for Garth Murray at https://www.eliteprospects.com/player/8956/garth-murray
Successfully scraped stats for Garth Murray
Failed rate: 0.00
Sleep for 0.34 minutes to prevent getting blocked

 [3] Collecting stats for Noah Clarke at https://www.eliteprospects.com/player/10321/noah-clarke
Successfully scraped stats for Noah Clarke
Failed rate: 0.00
Sleep for 1.04 minutes to prevent getting blocked

 [4] Collecting stats for Pierre Hedin at https://www.eliteprospects.com/player/218/pierre-hedin
Successfully scraped stats for Pierre Hedin
Failed rate: 0.00
Sleep for 0.20 minutes to prevent getting blocked

 [5] Collecting stats for Mikko Luoma at https://www.eliteprospects.com/player/2702/mikko-luoma
Successfully scraped stats for Mikko Luoma
Failed rate: 0.00
S

### 07/29/2025 --

In [14]:
# Scrape players from 1300-1350
curr_batch_metadata = nhl_players_metadata[1300:1350]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Mike Pandolfo at https://www.eliteprospects.com/player/14421/mike-pandolfo
Ad or overlay is blocking dropdown for Mike Pandolfo. Trying to remove it...
Failed to remove overlay/ad.
Failed to get 'Regular Season + Postseason' table: Message: element click intercepted: Element <div class=" css-x1uf2d-control">...</div> is not clickable at point (1258, 19). Other element would receive the click: <aside class="AdSlot_centering__vHSRy AdSlot_panorama__Eaw_j LayoutAds_slotH__O7qAf">...</aside>
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
	GetHandleVerifier [0x0x1081a33+62339]
	GetHandleVerifier [0x0x1081a74+62404]
	(No symbol) [0x0xec2123]
	(No symbol) [0x0xf10bb0]
	(No symbol) [0x0xf0ef6a]
	(No symbol) [0x0xf0cac7]
	(No symbol) [0x0xf0bd7d]
	(No symbol) [0x0xf00515]
	(No symbol) [0x0xf2f3bc]
	(No symbol) [0

In [15]:
# Scrape players from 1350-1400
curr_batch_metadata = nhl_players_metadata[1350:1400]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Jay McClement at https://www.eliteprospects.com/player/9517/jay-mcclement
Successfully scraped stats for Jay McClement
Failed rate: 0.00
Sleep for 1.24 minutes to prevent getting blocked

 [2] Collecting stats for Corey Perry at https://www.eliteprospects.com/player/8541/corey-perry
Error scraping Corey Perry: HTTPConnectionPool(host='localhost', port=52194): Read timed out. (read timeout=120)
Failed to get stats for Corey Perry: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.76 seconds before trying the next player

 [3] Collecting stats for Dennis Wideman at https://www.eliteprospects.com/player/8826/dennis-wideman
Error scraping Dennis Wideman: HTTPConnectionPool(host='localhost', port=52776): Read timed out. (read timeout=120)
Failed to get stats for Dennis Wideman: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.81 seconds before trying the next player

 [4] Collecting stats for Jaroslav Balastik at https://www.eliteprospects.com/p

In [16]:
# Scrape players from 1400-1450
curr_batch_metadata = nhl_players_metadata[1400:1450]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Geoff Platt at https://www.eliteprospects.com/player/9469/geoff-platt
Successfully scraped stats for Geoff Platt
Failed rate: 0.00
Sleep for 1.44 minutes to prevent getting blocked

 [2] Collecting stats for Gilbert Brulé at https://www.eliteprospects.com/player/10235/gilbert-brule
Successfully scraped stats for Gilbert Brulé
Failed rate: 0.00
Sleep for 0.23 minutes to prevent getting blocked

 [3] Collecting stats for Noah Welch at https://www.eliteprospects.com/player/10323/noah-welch
Successfully scraped stats for Noah Welch
Failed rate: 0.00
Sleep for 0.20 minutes to prevent getting blocked

 [4] Collecting stats for Yan Stastny at https://www.eliteprospects.com/player/8799/yan-stastny
Successfully scraped stats for Yan Stastny
Failed rate: 0.00
Sleep for 1.39 minutes to prevent getting blocked

 [5] Collecting stats for Jakub Klepis at https://www.eliteprospects.com/player/9609/jakub-klepis
Successfully scraped stats for Jakub Klepis
Failed rate: 0.00
Sl

In [17]:
# Scrape players from 1450-1500
curr_batch_metadata = nhl_players_metadata[1450:1500]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Greg Jacina at https://www.eliteprospects.com/player/10248/greg-jacina
Error scraping Greg Jacina: HTTPConnectionPool(host='localhost', port=61419): Read timed out. (read timeout=120)
Failed to get stats for Greg Jacina: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.69 seconds before trying the next player

 [2] Collecting stats for Matt Ryan at https://www.eliteprospects.com/player/14334/matt-ryan
Successfully scraped stats for Matt Ryan
Failed rate: 0.50
Sleep for 0.49 minutes to prevent getting blocked

 [3] Collecting stats for Nick Tarnasky at https://www.eliteprospects.com/player/8846/nick-tarnasky
Successfully scraped stats for Nick Tarnasky
Failed rate: 0.33
Sleep for 1.92 minutes to prevent getting blocked

 [4] Collecting stats for Andy Wozniewski at https://www.eliteprospects.com/player/8543/andy-wozniewski
Successfully scraped stats for Andy Wozniewski
Failed rate: 0.25
Sleep for 0.42 minutes to prevent getting blocked

 [5] Collectin

In [18]:
# Scrape players from 1500-1550
curr_batch_metadata = nhl_players_metadata[1500:1550]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Jean-François Jacques at https://www.eliteprospects.com/player/9026/jean-francois-jacques
Successfully scraped stats for Jean-François Jacques
Failed rate: 0.00
Sleep for 1.16 minutes to prevent getting blocked

 [2] Collecting stats for Dan Smith at https://www.eliteprospects.com/player/11762/dan-smith
Successfully scraped stats for Dan Smith
Failed rate: 0.00
Sleep for 1.13 minutes to prevent getting blocked

 [3] Collecting stats for Jason Ryznar at https://www.eliteprospects.com/player/8921/jason-ryznar
Successfully scraped stats for Jason Ryznar
Failed rate: 0.00
Sleep for 0.97 minutes to prevent getting blocked

 [4] Collecting stats for Jean-Philippe Côté at https://www.eliteprospects.com/player/9123/jean-philippe-cote
Successfully scraped stats for Jean-Philippe Côté
Failed rate: 0.00
Sleep for 1.03 minutes to prevent getting blocked

 [5] Collecting stats for Steven Regier at https://www.eliteprospects.com/player/10876/steven-regier
Successfully scra

In [19]:
# Scrape players from 1550-1600
curr_batch_metadata = nhl_players_metadata[1550:1600]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Jussi Timonen at https://www.eliteprospects.com/player/3193/jussi-timonen
Error scraping Jussi Timonen: HTTPConnectionPool(host='localhost', port=57410): Read timed out. (read timeout=120)
Failed to get stats for Jussi Timonen: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.83 seconds before trying the next player

 [2] Collecting stats for Marc Methot at https://www.eliteprospects.com/player/10993/marc-methot
Error scraping Marc Methot: HTTPConnectionPool(host='localhost', port=58076): Read timed out. (read timeout=120)
Failed to get stats for Marc Methot: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.62 seconds before trying the next player

 [3] Collecting stats for Chris Conner at https://www.eliteprospects.com/player/10995/chris-conner
Error scraping Chris Conner: HTTPConnectionPool(host='localhost', port=58867): Read timed out. (read timeout=120)
Failed to get stats for Chris Conner: 'NoneType' object has no attribute 'to_csv'
S

In [20]:
# Scrape players from 1600-1650
curr_batch_metadata = nhl_players_metadata[1600:1650]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Rob Schremp at https://www.eliteprospects.com/player/8876/rob-schremp
Failed to get 'Regular Season + Postseason' table: HTTPConnectionPool(host='localhost', port=58106): Max retries exceeded with url: /session/cf25289cbf85839338fdc7595b580048/source (Caused by ReadTimeoutError("HTTPConnectionPool(host='localhost', port=58106): Read timed out. (read timeout=120)"))
No stats found for Rob Schremp
Failed to get stats for Rob Schremp: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.78 seconds before trying the next player

 [2] Collecting stats for Tomi Mäki at https://www.eliteprospects.com/player/3215/tomi-maki
Successfully scraped stats for Tomi Mäki
Failed rate: 0.50
Sleep for 1.97 minutes to prevent getting blocked

 [3] Collecting stats for Dave Bolland at https://www.eliteprospects.com/player/9193/dave-bolland
Error scraping Dave Bolland: HTTPConnectionPool(host='localhost', port=59491): Read timed out. (read timeout=120)
Failed to get stats fo

In [21]:
# Scrape players from 1650-1700
curr_batch_metadata = nhl_players_metadata[1650:1700]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for David Perron at https://www.eliteprospects.com/player/11123/david-perron
Successfully scraped stats for David Perron
Failed rate: 0.00
Sleep for 0.92 minutes to prevent getting blocked

 [2] Collecting stats for Sergei Kostitsyn at https://www.eliteprospects.com/player/9653/sergei-kostitsyn
Successfully scraped stats for Sergei Kostitsyn
Failed rate: 0.00
Sleep for 1.20 minutes to prevent getting blocked

 [3] Collecting stats for Milan Lucic at https://www.eliteprospects.com/player/11104/milan-lucic
Successfully scraped stats for Milan Lucic
Failed rate: 0.00
Sleep for 0.67 minutes to prevent getting blocked

 [4] Collecting stats for Daniel Winnik at https://www.eliteprospects.com/player/11480/daniel-winnik
Successfully scraped stats for Daniel Winnik
Failed rate: 0.00
Sleep for 1.95 minutes to prevent getting blocked

 [5] Collecting stats for Matt Niskanen at https://www.eliteprospects.com/player/9214/matt-niskanen
Successfully scraped stats for Matt Nisk

In [22]:
# Scrape players from 1700-1750
curr_batch_metadata = nhl_players_metadata[1700:1750]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Shawn Matthias at https://www.eliteprospects.com/player/9345/shawn-matthias
Successfully scraped stats for Shawn Matthias
Failed rate: 0.00
Sleep for 0.46 minutes to prevent getting blocked

 [2] Collecting stats for Jordan Smotherman at https://www.eliteprospects.com/player/11568/jordan-smotherman
Error scraping Jordan Smotherman: HTTPConnectionPool(host='localhost', port=55296): Read timed out. (read timeout=120)
Failed to get stats for Jordan Smotherman: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.87 seconds before trying the next player

 [3] Collecting stats for Clay Wilson at https://www.eliteprospects.com/player/11404/clay-wilson
Successfully scraped stats for Clay Wilson
Failed rate: 0.33
Sleep for 0.26 minutes to prevent getting blocked

 [4] Collecting stats for Chris Minard at https://www.eliteprospects.com/player/11349/chris-minard
Successfully scraped stats for Chris Minard
Failed rate: 0.25
Sleep for 1.31 minutes to prevent gettin

In [23]:
# Scrape players from 1750-1800
curr_batch_metadata = nhl_players_metadata[1750:1800]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Claude Giroux at https://www.eliteprospects.com/player/10437/claude-giroux
Successfully scraped stats for Claude Giroux
Failed rate: 0.00
Sleep for 1.85 minutes to prevent getting blocked

 [2] Collecting stats for Adam Berti at https://www.eliteprospects.com/player/9493/adam-berti
Successfully scraped stats for Adam Berti
Failed rate: 0.00
Sleep for 0.61 minutes to prevent getting blocked

 [3] Collecting stats for Justin Abdelkader at https://www.eliteprospects.com/player/10460/justin-abdelkader
Successfully scraped stats for Justin Abdelkader
Failed rate: 0.00
Sleep for 1.94 minutes to prevent getting blocked

 [4] Collecting stats for Marc-André Gragnani at https://www.eliteprospects.com/player/9356/marc-andre-gragnani
Successfully scraped stats for Marc-André Gragnani
Failed rate: 0.00
Sleep for 1.34 minutes to prevent getting blocked

 [5] Collecting stats for Alex Foster at https://www.eliteprospects.com/player/11549/alex-foster
Successfully scraped st

### 07/19/2025 -- Morning

In [30]:
# Scrape players from 1800-1850
curr_batch_metadata = nhl_players_metadata[1800:1850]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Luca Sbisa at https://www.eliteprospects.com/player/13946/luca-sbisa
Successfully scraped stats for Luca Sbisa
Failed rate: 0.00
Sleep for 1.98 minutes to prevent getting blocked

 [2] Collecting stats for Jamie McGinn at https://www.eliteprospects.com/player/9336/jamie-mcginn
Failed to get 'Regular Season + Postseason' table: HTTPConnectionPool(host='localhost', port=62966): Read timed out. (read timeout=120)
No stats found for Jamie McGinn
Failed to get stats for Jamie McGinn: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.68 seconds before trying the next player

 [3] Collecting stats for Chris Butler at https://www.eliteprospects.com/player/9206/chris-butler
Successfully scraped stats for Chris Butler
Failed rate: 0.33
Sleep for 1.04 minutes to prevent getting blocked

 [4] Collecting stats for Martins Karsums at https://www.eliteprospects.com/player/9238/martins-karsums
Successfully scraped stats for Martins Karsums
Failed rate: 0.25
Sleep fo

In [31]:
# Scrape players from 1850-1900
curr_batch_metadata = nhl_players_metadata[1850:1900]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Warren Peters at https://www.eliteprospects.com/player/11744/warren-peters
Successfully scraped stats for Warren Peters
Failed rate: 0.00
Sleep for 1.76 minutes to prevent getting blocked

 [2] Collecting stats for Matt Halischuk at https://www.eliteprospects.com/player/12560/matt-halischuk
Successfully scraped stats for Matt Halischuk
Failed rate: 0.00
Sleep for 1.38 minutes to prevent getting blocked

 [3] Collecting stats for Brian Salcido at https://www.eliteprospects.com/player/11406/brian-salcido
Successfully scraped stats for Brian Salcido
Failed rate: 0.00
Sleep for 1.50 minutes to prevent getting blocked

 [4] Collecting stats for Jakub Petruzalek at https://www.eliteprospects.com/player/9617/jakub-petruzalek
Error scraping Jakub Petruzalek: Message: 

Failed to get stats for Jakub Petruzalek: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.33 seconds before trying the next player

 [5] Collecting stats for David Schlemko at https://www.el

In [32]:
# Scrape players from 1900-1950
curr_batch_metadata = nhl_players_metadata[1900:1950]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Andrew MacDonald at https://www.eliteprospects.com/player/12406/andrew-macdonald
Error scraping Andrew MacDonald: HTTPConnectionPool(host='localhost', port=58127): Read timed out. (read timeout=120)
Failed to get stats for Andrew MacDonald: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.98 seconds before trying the next player

 [2] Collecting stats for Jay Beagle at https://www.eliteprospects.com/player/14825/jay-beagle
Successfully scraped stats for Jay Beagle
Failed rate: 0.50
Sleep for 1.84 minutes to prevent getting blocked

 [3] Collecting stats for Per Ledin at https://www.eliteprospects.com/player/186/per-ledin
Error scraping Per Ledin: HTTPConnectionPool(host='localhost', port=59338): Read timed out. (read timeout=120)
Failed to get stats for Per Ledin: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.48 seconds before trying the next player

 [4] Collecting stats for Wes O'Neill at https://www.eliteprospects.com/player/9480/wes

In [33]:
# Scrape players from 1950-2000
curr_batch_metadata = nhl_players_metadata[1950:2000]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for John Carlson at https://www.eliteprospects.com/player/18590/john-carlson
Successfully scraped stats for John Carlson
Failed rate: 0.00
Sleep for 0.51 minutes to prevent getting blocked

 [2] Collecting stats for Frazer McLaren at https://www.eliteprospects.com/player/14501/frazer-mclaren
Successfully scraped stats for Frazer McLaren
Failed rate: 0.00
Sleep for 1.73 minutes to prevent getting blocked

 [3] Collecting stats for Benn Ferriero at https://www.eliteprospects.com/player/9378/benn-ferriero
Successfully scraped stats for Benn Ferriero
Failed rate: 0.00
Sleep for 1.59 minutes to prevent getting blocked

 [4] Collecting stats for Tom Pyatt at https://www.eliteprospects.com/player/9199/tom-pyatt
Successfully scraped stats for Tom Pyatt
Failed rate: 0.00
Sleep for 1.80 minutes to prevent getting blocked

 [5] Collecting stats for James Wright at https://www.eliteprospects.com/player/18595/james-wright
Successfully scraped stats for James Wright
Failed rat

### 07/20/2025 -- Midnight

In [34]:
# Scrape players from 2000-2050
curr_batch_metadata = nhl_players_metadata[2000:2050]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Eric Tangradi at https://www.eliteprospects.com/player/12514/eric-tangradi
Successfully scraped stats for Eric Tangradi
Failed rate: 0.00
Sleep for 0.52 minutes to prevent getting blocked

 [2] Collecting stats for Maxim Noreau at https://www.eliteprospects.com/player/12153/maxim-noreau
Successfully scraped stats for Maxim Noreau
Failed rate: 0.00
Sleep for 0.78 minutes to prevent getting blocked

 [3] Collecting stats for Andrei Loktionov at https://www.eliteprospects.com/player/11867/andrei-loktionov
Successfully scraped stats for Andrei Loktionov
Failed rate: 0.00
Sleep for 0.75 minutes to prevent getting blocked

 [4] Collecting stats for Marc-André Cliche at https://www.eliteprospects.com/player/10435/marc-andre-cliche
Successfully scraped stats for Marc-André Cliche
Failed rate: 0.00
Sleep for 0.72 minutes to prevent getting blocked

 [5] Collecting stats for Scott Jackson at https://www.eliteprospects.com/player/11619/scott-jackson
Successfully scraped

In [35]:
# Scrape players from 2050-2100
curr_batch_metadata = nhl_players_metadata[2050:2100]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Magnus Pääjärvi at https://www.eliteprospects.com/player/6014/magnus-paajarvi
Successfully scraped stats for Magnus Pääjärvi
Failed rate: 0.00
Sleep for 1.44 minutes to prevent getting blocked

 [2] Collecting stats for Marcus Johansson at https://www.eliteprospects.com/player/6225/marcus-johansson
Successfully scraped stats for Marcus Johansson
Failed rate: 0.00
Sleep for 0.22 minutes to prevent getting blocked

 [3] Collecting stats for Linus Omark at https://www.eliteprospects.com/player/4008/linus-omark
Successfully scraped stats for Linus Omark
Failed rate: 0.00
Sleep for 1.87 minutes to prevent getting blocked

 [4] Collecting stats for Travis Hamonic at https://www.eliteprospects.com/player/17519/travis-hamonic
Successfully scraped stats for Travis Hamonic
Failed rate: 0.00
Sleep for 1.60 minutes to prevent getting blocked

 [5] Collecting stats for Mats Zuccarello at https://www.eliteprospects.com/player/10881/mats-zuccarello
Error scraping Mats Zucca

In [6]:
# Scrape players from 2100-2150
curr_batch_metadata = nhl_players_metadata[2100:2150]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Joe Vitale at https://www.eliteprospects.com/player/15069/joe-vitale
Successfully scraped stats for Joe Vitale
Failed rate: 0.00
Sleep for 1.40 minutes to prevent getting blocked

 [2] Collecting stats for Aaron Volpatti at https://www.eliteprospects.com/player/39059/aaron-volpatti
Successfully scraped stats for Aaron Volpatti
Failed rate: 0.00
Sleep for 1.04 minutes to prevent getting blocked

 [3] Collecting stats for Patrice Cormier at https://www.eliteprospects.com/player/19042/patrice-cormier
Successfully scraped stats for Patrice Cormier
Failed rate: 0.00
Sleep for 1.54 minutes to prevent getting blocked

 [4] Collecting stats for Tomas Vincour at https://www.eliteprospects.com/player/12565/tomas-vincour
Successfully scraped stats for Tomas Vincour
Failed rate: 0.00
Sleep for 1.41 minutes to prevent getting blocked

 [5] Collecting stats for Tomas Kubalik at https://www.eliteprospects.com/player/11834/tomas-kubalik
Ad or overlay is blocking dropdown for

In [7]:
# Scrape players from 2150-2200
curr_batch_metadata = nhl_players_metadata[2150:2200]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Hugh Jessiman at https://www.eliteprospects.com/player/11748/hugh-jessiman
Successfully scraped stats for Hugh Jessiman
Failed rate: 0.00
Sleep for 0.71 minutes to prevent getting blocked

 [2] Collecting stats for Travis Morin at https://www.eliteprospects.com/player/11446/travis-morin
Successfully scraped stats for Travis Morin
Failed rate: 0.00
Sleep for 0.64 minutes to prevent getting blocked

 [3] Collecting stats for Nick Drazenovic at https://www.eliteprospects.com/player/11637/nick-drazenovic
Error scraping Nick Drazenovic: HTTPConnectionPool(host='localhost', port=57421): Read timed out. (read timeout=120)
Failed to get stats for Nick Drazenovic: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.72 seconds before trying the next player

 [4] Collecting stats for Andreas Engqvist at https://www.eliteprospects.com/player/6626/andreas-engqvist
Error scraping Andreas Engqvist: HTTPConnectionPool(host='localhost', port=58085): Read timed out. (re

In [16]:
# Scrape players from 2200-2250
curr_batch_metadata = nhl_players_metadata[2200:2250]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Louis Leblanc at https://www.eliteprospects.com/player/31615/louis-leblanc
Successfully scraped stats for Louis Leblanc
Failed rate: 0.00
Sleep for 1.45 minutes to prevent getting blocked

 [2] Collecting stats for Zack Kassian at https://www.eliteprospects.com/player/15390/zack-kassian
Successfully scraped stats for Zack Kassian
Failed rate: 0.00
Sleep for 0.52 minutes to prevent getting blocked

 [3] Collecting stats for David Savard at https://www.eliteprospects.com/player/32878/david-savard
Successfully scraped stats for David Savard
Failed rate: 0.00
Sleep for 1.56 minutes to prevent getting blocked

 [4] Collecting stats for Jimmy Hayes at https://www.eliteprospects.com/player/11463/jimmy-hayes
Successfully scraped stats for Jimmy Hayes
Failed rate: 0.00
Sleep for 1.19 minutes to prevent getting blocked

 [5] Collecting stats for Lennart Petrell at https://www.eliteprospects.com/player/4279/lennart-petrell
Successfully scraped stats for Lennart Petrell


In [17]:
# Scrape players from 2250-2300
curr_batch_metadata = nhl_players_metadata[2250:2300]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Cade Fairchild at https://www.eliteprospects.com/player/9309/cade-fairchild
Successfully scraped stats for Cade Fairchild
Failed rate: 0.00
Sleep for 0.23 minutes to prevent getting blocked

 [2] Collecting stats for Riley Nash at https://www.eliteprospects.com/player/12507/riley-nash
Successfully scraped stats for Riley Nash
Failed rate: 0.00
Sleep for 0.71 minutes to prevent getting blocked

 [3] Collecting stats for Mika Zibanejad at https://www.eliteprospects.com/player/23856/mika-zibanejad
Successfully scraped stats for Mika Zibanejad
Failed rate: 0.00
Sleep for 0.76 minutes to prevent getting blocked

 [4] Collecting stats for Colten Teubert at https://www.eliteprospects.com/player/13625/colten-teubert
Successfully scraped stats for Colten Teubert
Failed rate: 0.00
Sleep for 1.97 minutes to prevent getting blocked

 [5] Collecting stats for Dylan Olsen at https://www.eliteprospects.com/player/19294/dylan-olsen
Successfully scraped stats for Dylan Olsen


In [18]:
# Scrape players from 2300-2350
curr_batch_metadata = nhl_players_metadata[2300:2350]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Kevin Marshall at https://www.eliteprospects.com/player/12508/kevin-marshall
Successfully scraped stats for Kevin Marshall
Failed rate: 0.00
Sleep for 0.79 minutes to prevent getting blocked

 [2] Collecting stats for Carter Ashton at https://www.eliteprospects.com/player/21033/carter-ashton
Successfully scraped stats for Carter Ashton
Failed rate: 0.00
Sleep for 1.19 minutes to prevent getting blocked

 [3] Collecting stats for Brandon Bollig at https://www.eliteprospects.com/player/39199/brandon-bollig
Successfully scraped stats for Brandon Bollig
Failed rate: 0.00
Sleep for 0.57 minutes to prevent getting blocked

 [4] Collecting stats for Nail Yakupov at https://www.eliteprospects.com/player/21228/nail-yakupov
Successfully scraped stats for Nail Yakupov
Failed rate: 0.00
Sleep for 0.78 minutes to prevent getting blocked

 [5] Collecting stats for Jonathan Huberdeau at https://www.eliteprospects.com/player/45261/jonathan-huberdeau
Successfully scraped stat

In [19]:
# Scrape players from 2350-2400
curr_batch_metadata = nhl_players_metadata[2350:2400]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Dave Dziurzynski at https://www.eliteprospects.com/player/45871/dave-dziurzynski
Error scraping Dave Dziurzynski: HTTPConnectionPool(host='localhost', port=63570): Read timed out. (read timeout=120)
Failed to get stats for Dave Dziurzynski: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.67 seconds before trying the next player

 [2] Collecting stats for Anders Lee at https://www.eliteprospects.com/player/19068/anders-lee
Successfully scraped stats for Anders Lee
Failed rate: 0.50
Sleep for 0.39 minutes to prevent getting blocked

 [3] Collecting stats for Matt Tennyson at https://www.eliteprospects.com/player/37796/matt-tennyson
Successfully scraped stats for Matt Tennyson
Failed rate: 0.33
Sleep for 1.60 minutes to prevent getting blocked

 [4] Collecting stats for Anthony Peluso at https://www.eliteprospects.com/player/15038/anthony-peluso
Successfully scraped stats for Anthony Peluso
Failed rate: 0.25
Sleep for 1.12 minutes to prevent getting b

In [20]:
# Scrape players from 2400-2450
curr_batch_metadata = nhl_players_metadata[2400:2450]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Jon Rheault at https://www.eliteprospects.com/player/14861/jon-rheault
Successfully scraped stats for Jon Rheault
Failed rate: 0.00
Sleep for 0.82 minutes to prevent getting blocked

 [2] Collecting stats for Derek Grant at https://www.eliteprospects.com/player/18242/derek-grant
Successfully scraped stats for Derek Grant
Failed rate: 0.00
Sleep for 0.32 minutes to prevent getting blocked

 [3] Collecting stats for Scott Laughton at https://www.eliteprospects.com/player/76690/scott-laughton
Successfully scraped stats for Scott Laughton
Failed rate: 0.00
Sleep for 1.42 minutes to prevent getting blocked

 [4] Collecting stats for Chris Brown at https://www.eliteprospects.com/player/31273/chris-brown
Error scraping Chris Brown: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors

In [ ]:
# Scrape players from 2450-2500
curr_batch_metadata = nhl_players_metadata[2450:2500]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Linden Vey at https://www.eliteprospects.com/player/28422/linden-vey
Successfully scraped stats for Linden Vey
Failed rate: 0.00
Sleep for 0.32 minutes to prevent getting blocked

 [2] Collecting stats for Rasmus Ristolainen at https://www.eliteprospects.com/player/43497/rasmus-ristolainen
Successfully scraped stats for Rasmus Ristolainen
Failed rate: 0.00
Sleep for 1.46 minutes to prevent getting blocked

 [3] Collecting stats for Luke Gazdic at https://www.eliteprospects.com/player/15037/luke-gazdic
Successfully scraped stats for Luke Gazdic
Failed rate: 0.00
Sleep for 1.24 minutes to prevent getting blocked

 [4] Collecting stats for Colton Sissons at https://www.eliteprospects.com/player/48393/colton-sissons
Successfully scraped stats for Colton Sissons
Failed rate: 0.00
Sleep for 1.78 minutes to prevent getting blocked

 [5] Collecting stats for Michael Latta at https://www.eliteprospects.com/player/14436/michael-latta
Successfully scraped stats for Mich

### 07/21/2025 -- Midnight

In [16]:
# Scrape players from 2500-2550
curr_batch_metadata = nhl_players_metadata[2500:2550]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Petter Granberg at https://www.eliteprospects.com/player/13901/petter-granberg
Successfully scraped stats for Petter Granberg
Failed rate: 0.00
Sleep for 0.35 minutes to prevent getting blocked

 [2] Collecting stats for Julian Melchiori at https://www.eliteprospects.com/player/73778/julian-melchiori
Successfully scraped stats for Julian Melchiori
Failed rate: 0.00
Sleep for 0.78 minutes to prevent getting blocked

 [3] Collecting stats for Ryan Sproul at https://www.eliteprospects.com/player/62407/ryan-sproul
Successfully scraped stats for Ryan Sproul
Failed rate: 0.00
Sleep for 1.74 minutes to prevent getting blocked

 [4] Collecting stats for Cody Kunyk at https://www.eliteprospects.com/player/44471/cody-kunyk
Successfully scraped stats for Cody Kunyk
Failed rate: 0.00
Sleep for 1.70 minutes to prevent getting blocked

 [5] Collecting stats for Bill Arnold at https://www.eliteprospects.com/player/40260/bill-arnold
Successfully scraped stats for Bill Arnold

In [17]:
# Scrape players from 2550-2600
curr_batch_metadata = nhl_players_metadata[2550:2600]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Victor Rask at https://www.eliteprospects.com/player/15747/victor-rask
Successfully scraped stats for Victor Rask
Failed rate: 0.00
Sleep for 0.51 minutes to prevent getting blocked

 [2] Collecting stats for Jonathan Drouin at https://www.eliteprospects.com/player/118657/jonathan-drouin
Successfully scraped stats for Jonathan Drouin
Failed rate: 0.00
Sleep for 1.22 minutes to prevent getting blocked

 [3] Collecting stats for David Pastrnak at https://www.eliteprospects.com/player/130383/david-pastrnak
Successfully scraped stats for David Pastrnak
Failed rate: 0.00
Sleep for 1.25 minutes to prevent getting blocked

 [4] Collecting stats for Bo Horvat at https://www.eliteprospects.com/player/117012/bo-horvat
Successfully scraped stats for Bo Horvat
Failed rate: 0.00
Sleep for 0.62 minutes to prevent getting blocked

 [5] Collecting stats for Melker Karlsson at https://www.eliteprospects.com/player/10014/melker-karlsson
Successfully scraped stats for Melker Ka

In [18]:
# Scrape players from 2600-2650
curr_batch_metadata = nhl_players_metadata[2600:2650]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Tim Schaller at https://www.eliteprospects.com/player/38059/tim-schaller
Successfully scraped stats for Tim Schaller
Failed rate: 0.00
Sleep for 0.38 minutes to prevent getting blocked

 [2] Collecting stats for Petr Straka at https://www.eliteprospects.com/player/34805/petr-straka
Successfully scraped stats for Petr Straka
Failed rate: 0.00
Sleep for 1.26 minutes to prevent getting blocked

 [3] Collecting stats for David Musil at https://www.eliteprospects.com/player/23298/david-musil
Successfully scraped stats for David Musil
Failed rate: 0.00
Sleep for 1.00 minutes to prevent getting blocked

 [4] Collecting stats for Danny Biega at https://www.eliteprospects.com/player/40194/danny-biega
Successfully scraped stats for Danny Biega
Failed rate: 0.00
Sleep for 0.43 minutes to prevent getting blocked

 [5] Collecting stats for Andrew MacWilliam at https://www.eliteprospects.com/player/19165/andrew-macwilliam
Successfully scraped stats for Andrew MacWilliam
Fa

In [19]:
# Scrape players from 2650-2700
curr_batch_metadata = nhl_players_metadata[2650:2700]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Tyler Gaudet at https://www.eliteprospects.com/player/96052/tyler-gaudet
Successfully scraped stats for Tyler Gaudet
Failed rate: 0.00
Sleep for 1.96 minutes to prevent getting blocked

 [2] Collecting stats for Brendan Woods at https://www.eliteprospects.com/player/45678/brendan-woods
Successfully scraped stats for Brendan Woods
Failed rate: 0.00
Sleep for 0.65 minutes to prevent getting blocked

 [3] Collecting stats for Phillip Danault at https://www.eliteprospects.com/player/45282/phillip-danault
Successfully scraped stats for Phillip Danault
Failed rate: 0.00
Sleep for 0.83 minutes to prevent getting blocked

 [4] Collecting stats for Dominik Uher at https://www.eliteprospects.com/player/41098/dominik-uher
Error scraping Dominik Uher: HTTPConnectionPool(host='localhost', port=60052): Read timed out. (read timeout=120)
Failed to get stats for Dominik Uher: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.66 seconds before trying the next player


In [6]:
# Scrape players from 2700-2750
curr_batch_metadata = nhl_players_metadata[2700:2750]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Andreas Martinsen at https://www.eliteprospects.com/player/11958/andreas-martinsen
Successfully scraped stats for Andreas Martinsen
Failed rate: 0.00
Sleep for 0.41 minutes to prevent getting blocked

 [2] Collecting stats for Conor Sheary at https://www.eliteprospects.com/player/94484/conor-sheary
Successfully scraped stats for Conor Sheary
Failed rate: 0.00
Sleep for 1.90 minutes to prevent getting blocked

 [3] Collecting stats for Daniel Carr at https://www.eliteprospects.com/player/39435/daniel-carr
Successfully scraped stats for Daniel Carr
Failed rate: 0.00
Sleep for 0.80 minutes to prevent getting blocked

 [4] Collecting stats for Dennis Rasmussen at https://www.eliteprospects.com/player/9985/dennis-rasmussen
Successfully scraped stats for Dennis Rasmussen
Failed rate: 0.00
Sleep for 0.79 minutes to prevent getting blocked

 [5] Collecting stats for Ryan Dzingel at https://www.eliteprospects.com/player/49315/ryan-dzingel
Successfully scraped stats fo

In [7]:
# Scrape players from 2750-2800
curr_batch_metadata = nhl_players_metadata[2750:2800]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Pavel Zacha at https://www.eliteprospects.com/player/130786/pavel-zacha
Successfully scraped stats for Pavel Zacha
Failed rate: 0.00
Sleep for 1.25 minutes to prevent getting blocked

 [2] Collecting stats for Charles Hudon at https://www.eliteprospects.com/player/76324/charles-hudon
Successfully scraped stats for Charles Hudon
Failed rate: 0.00
Sleep for 1.78 minutes to prevent getting blocked

 [3] Collecting stats for Viktor Lööv at https://www.eliteprospects.com/player/20058/viktor-loov
Successfully scraped stats for Viktor Lööv
Failed rate: 0.00
Sleep for 0.62 minutes to prevent getting blocked

 [4] Collecting stats for Tobias Lindberg at https://www.eliteprospects.com/player/85671/tobias-lindberg
Successfully scraped stats for Tobias Lindberg
Failed rate: 0.00
Sleep for 1.53 minutes to prevent getting blocked

 [5] Collecting stats for Adam Pelech at https://www.eliteprospects.com/player/89694/adam-pelech
Successfully scraped stats for Adam Pelech
Fail

In [8]:
# Scrape players from 2800-2850
curr_batch_metadata = nhl_players_metadata[2800:2850]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Mike Matheson at https://www.eliteprospects.com/player/76333/mike-matheson
Successfully scraped stats for Mike Matheson
Failed rate: 0.00
Sleep for 0.65 minutes to prevent getting blocked

 [2] Collecting stats for Kurtis Gabriel at https://www.eliteprospects.com/player/92683/kurtis-gabriel
Failed to get 'Regular Season + Postseason' table: HTTPConnectionPool(host='localhost', port=62792): Max retries exceeded with url: /session/ab3f1789c9abca42538c2430ed6e45c9/source (Caused by ReadTimeoutError("HTTPConnectionPool(host='localhost', port=62792): Read timed out. (read timeout=120)"))
No stats found for Kurtis Gabriel
Failed to get stats for Kurtis Gabriel: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.71 seconds before trying the next player

 [3] Collecting stats for Markus Hännikäinen at https://www.eliteprospects.com/player/45426/markus-hannikainen
Successfully scraped stats for Markus Hännikäinen
Failed rate: 0.33
Sleep for 0.64 minutes to pre

In [9]:
# Scrape players from 2850-2900
curr_batch_metadata = nhl_players_metadata[2850:2900]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Austin Czarnik at https://www.eliteprospects.com/player/20707/austin-czarnik
Successfully scraped stats for Austin Czarnik
Failed rate: 0.00
Sleep for 1.62 minutes to prevent getting blocked

 [2] Collecting stats for Nick Jensen at https://www.eliteprospects.com/player/37773/nick-jensen
Successfully scraped stats for Nick Jensen
Failed rate: 0.00
Sleep for 1.79 minutes to prevent getting blocked

 [3] Collecting stats for Ivan Barbashyov at https://www.eliteprospects.com/player/108659/ivan-barbashyov
Successfully scraped stats for Ivan Barbashyov
Failed rate: 0.00
Sleep for 1.86 minutes to prevent getting blocked

 [4] Collecting stats for Lawson Crouse at https://www.eliteprospects.com/player/205155/lawson-crouse
Successfully scraped stats for Lawson Crouse
Failed rate: 0.00
Sleep for 1.65 minutes to prevent getting blocked

 [5] Collecting stats for Jake Dotchin at https://www.eliteprospects.com/player/96063/jake-dotchin
Successfully scraped stats for Jake

In [10]:
# Scrape players from 2900-2950
curr_batch_metadata = nhl_players_metadata[2900:2950]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Wade Megan at https://www.eliteprospects.com/player/38054/wade-megan
Failed to get 'Regular Season + Postseason' table: HTTPConnectionPool(host='localhost', port=58970): Max retries exceeded with url: /session/f7f355e01b4192b97b414b1b85942260/source (Caused by ReadTimeoutError("HTTPConnectionPool(host='localhost', port=58970): Read timed out. (read timeout=120)"))
No stats found for Wade Megan
Failed to get stats for Wade Megan: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.44 seconds before trying the next player

 [2] Collecting stats for Danny O'Regan at https://www.eliteprospects.com/player/121936/danny-o-regan
Successfully scraped stats for Danny O'Regan
Failed rate: 0.50
Sleep for 0.30 minutes to prevent getting blocked

 [3] Collecting stats for Tyson Jost at https://www.eliteprospects.com/player/148111/tyson-jost
Successfully scraped stats for Tyson Jost
Failed rate: 0.33
Sleep for 1.07 minutes to prevent getting blocked

 [4] Collecting 

In [11]:
# Scrape players from 2950-3000
curr_batch_metadata = nhl_players_metadata[2950:3000]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Tanner Richard at https://www.eliteprospects.com/player/41755/tanner-richard
Successfully scraped stats for Tanner Richard
Failed rate: 0.00
Sleep for 1.46 minutes to prevent getting blocked

 [2] Collecting stats for MacKenzie Weegar at https://www.eliteprospects.com/player/97908/mackenzie-weegar
Successfully scraped stats for MacKenzie Weegar
Failed rate: 0.00
Sleep for 1.15 minutes to prevent getting blocked

 [3] Collecting stats for Ben Thomson at https://www.eliteprospects.com/player/44664/ben-thomson
Successfully scraped stats for Ben Thomson
Failed rate: 0.00
Sleep for 1.92 minutes to prevent getting blocked

 [4] Collecting stats for Chris DiDomenico at https://www.eliteprospects.com/player/11319/chris-didomenico
Successfully scraped stats for Chris DiDomenico
Failed rate: 0.00
Sleep for 0.44 minutes to prevent getting blocked

 [5] Collecting stats for Mikhail Sergachyov at https://www.eliteprospects.com/player/312213/mikhail-sergachyov
Successfully

### 07/22/2025 -- Midnight

In [12]:
# Scrape players from 3000-3050
curr_batch_metadata = nhl_players_metadata[3000:3050]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Tanner Fritz at https://www.eliteprospects.com/player/23940/tanner-fritz
Successfully scraped stats for Tanner Fritz
Failed rate: 0.00
Sleep for 1.71 minutes to prevent getting blocked

 [2] Collecting stats for Victor Mete at https://www.eliteprospects.com/player/232703/victor-mete
Successfully scraped stats for Victor Mete
Failed rate: 0.00
Sleep for 0.19 minutes to prevent getting blocked

 [3] Collecting stats for Zach Aston-Reese at https://www.eliteprospects.com/player/98068/zach-aston-reese
Successfully scraped stats for Zach Aston-Reese
Failed rate: 0.00
Sleep for 0.92 minutes to prevent getting blocked

 [4] Collecting stats for Oskar Lindblom at https://www.eliteprospects.com/player/86181/oskar-lindblom
Ad or overlay is blocking dropdown for Oskar Lindblom. Trying to remove it...
Successfully scraped stats for Oskar Lindblom
Failed rate: 0.00
Sleep for 1.24 minutes to prevent getting blocked

 [5] Collecting stats for John Gilmour at https://www.eli

In [13]:
# Scrape players from 3050-3100
curr_batch_metadata = nhl_players_metadata[3050:3100]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Victor Ejdsell at https://www.eliteprospects.com/player/29343/victor-ejdsell
Successfully scraped stats for Victor Ejdsell
Failed rate: 0.00
Sleep for 0.58 minutes to prevent getting blocked

 [2] Collecting stats for Jordan Greenway at https://www.eliteprospects.com/player/183455/jordan-greenway
Failed to get 'Regular Season + Postseason' table: HTTPConnectionPool(host='localhost', port=58324): Max retries exceeded with url: /session/5a2b9c2a921459941be19feee6c80073/source (Caused by ReadTimeoutError("HTTPConnectionPool(host='localhost', port=58324): Read timed out. (read timeout=120)"))
No stats found for Jordan Greenway
Failed to get stats for Jordan Greenway: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.44 seconds before trying the next player

 [3] Collecting stats for Dillon Heatherington at https://www.eliteprospects.com/player/62233/dillon-heatherington
Successfully scraped stats for Dillon Heatherington
Failed rate: 0.33
Sleep for 0.82 

In [14]:
# Scrape players from 3100-3150
curr_batch_metadata = nhl_players_metadata[3100:3150]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Mathieu Joseph at https://www.eliteprospects.com/player/227640/mathieu-joseph
Successfully scraped stats for Mathieu Joseph
Failed rate: 0.00
Sleep for 0.29 minutes to prevent getting blocked

 [2] Collecting stats for Brett Howden at https://www.eliteprospects.com/player/232712/brett-howden
Successfully scraped stats for Brett Howden
Failed rate: 0.00
Sleep for 0.49 minutes to prevent getting blocked

 [3] Collecting stats for Filip Hronek at https://www.eliteprospects.com/player/191286/filip-hronek
Successfully scraped stats for Filip Hronek
Failed rate: 0.00
Sleep for 0.28 minutes to prevent getting blocked

 [4] Collecting stats for Roope Hintz at https://www.eliteprospects.com/player/92117/roope-hintz
Successfully scraped stats for Roope Hintz
Failed rate: 0.00
Sleep for 1.65 minutes to prevent getting blocked

 [5] Collecting stats for Austin Wagner at https://www.eliteprospects.com/player/161117/austin-wagner
Successfully scraped stats for Austin Wagne

In [15]:
# Scrape players from 3150-3200
curr_batch_metadata = nhl_players_metadata[3150:3200]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Dillon Dubé at https://www.eliteprospects.com/player/157256/dillon-dube
Successfully scraped stats for Dillon Dubé
Failed rate: 0.00
Sleep for 1.87 minutes to prevent getting blocked

 [2] Collecting stats for Victor Olofsson at https://www.eliteprospects.com/player/86308/victor-olofsson
Successfully scraped stats for Victor Olofsson
Failed rate: 0.00
Sleep for 0.66 minutes to prevent getting blocked

 [3] Collecting stats for Max Véronneau at https://www.eliteprospects.com/player/206777/max-veronneau
Successfully scraped stats for Max Véronneau
Failed rate: 0.00
Sleep for 1.08 minutes to prevent getting blocked

 [4] Collecting stats for Jonas Siegenthaler at https://www.eliteprospects.com/player/105407/jonas-siegenthaler
Successfully scraped stats for Jonas Siegenthaler
Failed rate: 0.00
Sleep for 1.06 minutes to prevent getting blocked

 [5] Collecting stats for Ilya Lyubushkin at https://www.eliteprospects.com/player/123109/ilya-lyubushkin
Successfully sc

In [16]:
# Scrape players from 3200-3250
curr_batch_metadata = nhl_players_metadata[3200:3250]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Riley Stillman at https://www.eliteprospects.com/player/252469/riley-stillman
Successfully scraped stats for Riley Stillman
Failed rate: 0.00
Sleep for 1.05 minutes to prevent getting blocked

 [2] Collecting stats for Rem Pitlick at https://www.eliteprospects.com/player/199968/rem-pitlick
Successfully scraped stats for Rem Pitlick
Failed rate: 0.00
Sleep for 1.81 minutes to prevent getting blocked

 [3] Collecting stats for Josh Teves at https://www.eliteprospects.com/player/197849/josh-teves
Successfully scraped stats for Josh Teves
Failed rate: 0.00
Sleep for 1.78 minutes to prevent getting blocked

 [4] Collecting stats for Erik Brännström at https://www.eliteprospects.com/player/255167/erik-brannstrom
Successfully scraped stats for Erik Brännström
Failed rate: 0.00
Sleep for 0.52 minutes to prevent getting blocked

 [5] Collecting stats for Brogan Rafferty at https://www.eliteprospects.com/player/248525/brogan-rafferty
Successfully scraped stats for Brog

In [17]:
# Scrape players from 3250-3300
curr_batch_metadata = nhl_players_metadata[3250:3300]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Gabriel Vilardi at https://www.eliteprospects.com/player/286867/gabriel-vilardi
Successfully scraped stats for Gabriel Vilardi
Failed rate: 0.00
Sleep for 0.34 minutes to prevent getting blocked

 [2] Collecting stats for Joel Kellman at https://www.eliteprospects.com/player/33719/joel-kellman
Successfully scraped stats for Joel Kellman
Failed rate: 0.00
Sleep for 0.72 minutes to prevent getting blocked

 [3] Collecting stats for Morgan Frost at https://www.eliteprospects.com/player/317025/morgan-frost
Successfully scraped stats for Morgan Frost
Failed rate: 0.00
Sleep for 0.97 minutes to prevent getting blocked

 [4] Collecting stats for Jansen Harkins at https://www.eliteprospects.com/player/160158/jansen-harkins
Successfully scraped stats for Jansen Harkins
Failed rate: 0.00
Sleep for 0.84 minutes to prevent getting blocked

 [5] Collecting stats for Noah Dobson at https://www.eliteprospects.com/player/275797/noah-dobson
Successfully scraped stats for Noah

In [18]:
# Scrape players from 3300-3350
curr_batch_metadata = nhl_players_metadata[3300:3350]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Tyler Benson at https://www.eliteprospects.com/player/154220/tyler-benson
Successfully scraped stats for Tyler Benson
Failed rate: 0.00
Sleep for 0.45 minutes to prevent getting blocked

 [2] Collecting stats for Eetu Luostarinen at https://www.eliteprospects.com/player/299319/eetu-luostarinen
Successfully scraped stats for Eetu Luostarinen
Failed rate: 0.00
Sleep for 0.56 minutes to prevent getting blocked

 [3] Collecting stats for Mathieu Olivier at https://www.eliteprospects.com/player/213604/mathieu-olivier
Successfully scraped stats for Mathieu Olivier
Failed rate: 0.00
Sleep for 0.45 minutes to prevent getting blocked

 [4] Collecting stats for Alexander Volkov at https://www.eliteprospects.com/player/277802/alexander-volkov
Successfully scraped stats for Alexander Volkov
Failed rate: 0.00
Sleep for 0.61 minutes to prevent getting blocked

 [5] Collecting stats for Ryan MacInnis at https://www.eliteprospects.com/player/168707/ryan-macinnis
Successfully

In [19]:
# Scrape players from 3350-3400
curr_batch_metadata = nhl_players_metadata[3350:3400]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Alexis Lafrenière at https://www.eliteprospects.com/player/296697/alexis-lafreniere
Successfully scraped stats for Alexis Lafrenière
Failed rate: 0.00
Sleep for 0.60 minutes to prevent getting blocked

 [2] Collecting stats for Philipp Kurashev at https://www.eliteprospects.com/player/279238/philipp-kurashev
Successfully scraped stats for Philipp Kurashev
Failed rate: 0.00
Sleep for 0.67 minutes to prevent getting blocked

 [3] Collecting stats for Artyom Zub at https://www.eliteprospects.com/player/182436/artyom-zub
Successfully scraped stats for Artyom Zub
Failed rate: 0.00
Sleep for 1.60 minutes to prevent getting blocked

 [4] Collecting stats for Dylan Cozens at https://www.eliteprospects.com/player/286938/dylan-cozens
Successfully scraped stats for Dylan Cozens
Failed rate: 0.00
Sleep for 1.43 minutes to prevent getting blocked

 [5] Collecting stats for Trevor Zegras at https://www.eliteprospects.com/player/424453/trevor-zegras
Successfully scraped sta

In [20]:
# Scrape players from 3400-3450
curr_batch_metadata = nhl_players_metadata[3400:3450]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Joe Veleno at https://www.eliteprospects.com/player/271713/joe-veleno
Successfully scraped stats for Joe Veleno
Failed rate: 0.00
Sleep for 0.17 minutes to prevent getting blocked

 [2] Collecting stats for Rasmus Kupari at https://www.eliteprospects.com/player/319079/rasmus-kupari
Successfully scraped stats for Rasmus Kupari
Failed rate: 0.00
Sleep for 0.45 minutes to prevent getting blocked

 [3] Collecting stats for Fredrik Händemark at https://www.eliteprospects.com/player/27259/fredrik-handemark
Successfully scraped stats for Fredrik Händemark
Failed rate: 0.00
Sleep for 0.33 minutes to prevent getting blocked

 [4] Collecting stats for Dakota Joshua at https://www.eliteprospects.com/player/194948/dakota-joshua
Successfully scraped stats for Dakota Joshua
Failed rate: 0.00
Sleep for 1.44 minutes to prevent getting blocked

 [5] Collecting stats for Justin Richards at https://www.eliteprospects.com/player/272539/justin-richards
Successfully scraped stats 

In [21]:
# Scrape players from 3450-3500
curr_batch_metadata = nhl_players_metadata[3450:3500]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Simon Benoît at https://www.eliteprospects.com/player/254891/simon-benoit
Successfully scraped stats for Simon Benoît
Failed rate: 0.00
Sleep for 1.01 minutes to prevent getting blocked

 [2] Collecting stats for Josh Dunne at https://www.eliteprospects.com/player/272225/josh-dunne
Successfully scraped stats for Josh Dunne
Failed rate: 0.00
Sleep for 1.49 minutes to prevent getting blocked

 [3] Collecting stats for Kole Lind at https://www.eliteprospects.com/player/200933/kole-lind
Successfully scraped stats for Kole Lind
Failed rate: 0.00
Sleep for 0.41 minutes to prevent getting blocked

 [4] Collecting stats for Matt Kiersted at https://www.eliteprospects.com/player/291904/matt-kiersted
Successfully scraped stats for Matt Kiersted
Failed rate: 0.00
Sleep for 1.70 minutes to prevent getting blocked

 [5] Collecting stats for Jeffrey Viel at https://www.eliteprospects.com/player/213441/jeffrey-viel
Successfully scraped stats for Jeffrey Viel
Failed rate: 0.

In [22]:
# Scrape players from 3500-3550
curr_batch_metadata = nhl_players_metadata[3500:3550]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Thomas Bordeleau at https://www.eliteprospects.com/player/296636/thomas-bordeleau
Successfully scraped stats for Thomas Bordeleau
Failed rate: 0.00
Sleep for 0.57 minutes to prevent getting blocked

 [2] Collecting stats for Dylan Samberg at https://www.eliteprospects.com/player/201527/dylan-samberg
Successfully scraped stats for Dylan Samberg
Failed rate: 0.00
Sleep for 1.95 minutes to prevent getting blocked

 [3] Collecting stats for Lassi Thomson at https://www.eliteprospects.com/player/372919/lassi-thomson
Error scraping Lassi Thomson: HTTPConnectionPool(host='localhost', port=55265): Read timed out. (read timeout=120)
Failed to get stats for Lassi Thomson: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.99 seconds before trying the next player

 [4] Collecting stats for Ronnie Attard at https://www.eliteprospects.com/player/284030/ronnie-attard
Successfully scraped stats for Ronnie Attard
Failed rate: 0.25
Sleep for 1.95 minutes to prevent ge

In [6]:
# Scrape players from 3550-3600
curr_batch_metadata = nhl_players_metadata[3550:3600]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Valtteri Puustinen at https://www.eliteprospects.com/player/332963/valtteri-puustinen
Successfully scraped stats for Valtteri Puustinen
Failed rate: 0.00
Sleep for 0.85 minutes to prevent getting blocked

 [2] Collecting stats for Reilly Walsh at https://www.eliteprospects.com/player/300597/reilly-walsh
Successfully scraped stats for Reilly Walsh
Failed rate: 0.00
Sleep for 0.41 minutes to prevent getting blocked

 [3] Collecting stats for Lucas Johansen at https://www.eliteprospects.com/player/199937/lucas-johansen
Successfully scraped stats for Lucas Johansen
Failed rate: 0.00
Sleep for 1.00 minutes to prevent getting blocked

 [4] Collecting stats for Fredrik Karlström at https://www.eliteprospects.com/player/220059/fredrik-karlstrom
Successfully scraped stats for Fredrik Karlström
Failed rate: 0.00
Sleep for 1.80 minutes to prevent getting blocked

 [5] Collecting stats for Alex Steeves at https://www.eliteprospects.com/player/354353/alex-steeves
Successf

In [7]:
# Scrape players from 3600-3650
curr_batch_metadata = nhl_players_metadata[3600:3650]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Chase Priskie at https://www.eliteprospects.com/player/177666/chase-priskie
Successfully scraped stats for Chase Priskie
Failed rate: 0.00
Sleep for 0.55 minutes to prevent getting blocked

 [2] Collecting stats for Isaak Phillips at https://www.eliteprospects.com/player/416750/isaak-phillips
Successfully scraped stats for Isaak Phillips
Failed rate: 0.00
Sleep for 0.99 minutes to prevent getting blocked

 [3] Collecting stats for Adam Raska at https://www.eliteprospects.com/player/298159/adam-raska
Error scraping Adam Raska: HTTPConnectionPool(host='localhost', port=52293): Read timed out. (read timeout=120)
Failed to get stats for Adam Raska: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.97 seconds before trying the next player

 [4] Collecting stats for Jakub Galvas at https://www.eliteprospects.com/player/236429/jakub-galvas
Successfully scraped stats for Jakub Galvas
Failed rate: 0.25
Sleep for 1.84 minutes to prevent getting blocked

 [5] C

In [8]:
# Scrape players from 3650-3700
curr_batch_metadata = nhl_players_metadata[3650:3700]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Tyler Kleven at https://www.eliteprospects.com/player/536644/tyler-kleven
Successfully scraped stats for Tyler Kleven
Failed rate: 0.00
Sleep for 0.50 minutes to prevent getting blocked

 [2] Collecting stats for Spencer Stastney at https://www.eliteprospects.com/player/283918/spencer-stastney
Successfully scraped stats for Spencer Stastney
Failed rate: 0.00
Sleep for 1.21 minutes to prevent getting blocked

 [3] Collecting stats for Brandt Clarke at https://www.eliteprospects.com/player/336887/brandt-clarke
Successfully scraped stats for Brandt Clarke
Failed rate: 0.00
Sleep for 0.24 minutes to prevent getting blocked

 [4] Collecting stats for Hunter McKown at https://www.eliteprospects.com/player/417327/hunter-mckown
Error scraping Hunter McKown: HTTPConnectionPool(host='localhost', port=63304): Read timed out. (read timeout=120)
Failed to get stats for Hunter McKown: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.80 seconds before trying the n

In [9]:
# Scrape players from 3700-3750
curr_batch_metadata = nhl_players_metadata[3700:3750]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Filip Král at https://www.eliteprospects.com/player/247241/filip-kral
Successfully scraped stats for Filip Král
Failed rate: 0.00
Sleep for 1.19 minutes to prevent getting blocked

 [2] Collecting stats for Tristen Robins at https://www.eliteprospects.com/player/286940/tristen-robins
Successfully scraped stats for Tristen Robins
Failed rate: 0.00
Sleep for 0.86 minutes to prevent getting blocked

 [3] Collecting stats for Jonathan Gruden at https://www.eliteprospects.com/player/276187/jonathan-gruden
Error scraping Jonathan Gruden: HTTPConnectionPool(host='localhost', port=57717): Read timed out. (read timeout=120)
Failed to get stats for Jonathan Gruden: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.25 seconds before trying the next player

 [4] Collecting stats for Elliot Desnoyers at https://www.eliteprospects.com/player/290157/elliot-desnoyers
Successfully scraped stats for Elliot Desnoyers
Failed rate: 0.25
Sleep for 0.24 minutes to prevent 

In [10]:
# Scrape players from 3750-3800
curr_batch_metadata = nhl_players_metadata[3750:3800]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Ilya Solovyov at https://www.eliteprospects.com/player/490245/ilya-solovyov
Successfully scraped stats for Ilya Solovyov
Failed rate: 0.00
Sleep for 1.61 minutes to prevent getting blocked

 [2] Collecting stats for Louis Crevier at https://www.eliteprospects.com/player/410534/louis-crevier
Successfully scraped stats for Louis Crevier
Failed rate: 0.00
Sleep for 1.37 minutes to prevent getting blocked

 [3] Collecting stats for Adam Edström at https://www.eliteprospects.com/player/349951/adam-edstrom
Successfully scraped stats for Adam Edström
Failed rate: 0.00
Sleep for 0.86 minutes to prevent getting blocked

 [4] Collecting stats for Vasili Ponomaryov at https://www.eliteprospects.com/player/448947/vasili-ponomaryov
Successfully scraped stats for Vasili Ponomaryov
Failed rate: 0.00
Sleep for 1.18 minutes to prevent getting blocked

 [5] Collecting stats for Liam Öhgren at https://www.eliteprospects.com/player/535698/liam-ohgren
Successfully scraped stats f

In [11]:
# Scrape players from 3800-3850
curr_batch_metadata = nhl_players_metadata[3800:3850]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Ondřej Pavel at https://www.eliteprospects.com/player/236950/ond%C5%99ej-pavel
Successfully scraped stats for Ondřej Pavel
Failed rate: 0.00
Sleep for 1.39 minutes to prevent getting blocked

 [2] Collecting stats for Gage Goncalves at https://www.eliteprospects.com/player/407591/gage-goncalves
Error scraping Gage Goncalves: HTTPConnectionPool(host='localhost', port=52687): Read timed out. (read timeout=120)
Failed to get stats for Gage Goncalves: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.83 seconds before trying the next player

 [3] Collecting stats for Justin Sourdif at https://www.eliteprospects.com/player/290346/justin-sourdif
Successfully scraped stats for Justin Sourdif
Failed rate: 0.33
Sleep for 0.41 minutes to prevent getting blocked

 [4] Collecting stats for Jack Thompson at https://www.eliteprospects.com/player/201878/jack-thompson
Successfully scraped stats for Jack Thompson
Failed rate: 0.25
Sleep for 1.54 minutes to prevent ge

In [12]:
# Scrape players from 3850-3900
curr_batch_metadata = nhl_players_metadata[3850:3900]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Sam Rinzel at https://www.eliteprospects.com/player/670126/sam-rinzel
Successfully scraped stats for Sam Rinzel
Failed rate: 0.00
Sleep for 1.23 minutes to prevent getting blocked

 [2] Collecting stats for Jimmy Snuggerud at https://www.eliteprospects.com/player/623832/jimmy-snuggerud
Error scraping Jimmy Snuggerud: HTTPConnectionPool(host='localhost', port=50569): Read timed out. (read timeout=120)
Failed to get stats for Jimmy Snuggerud: 'NoneType' object has no attribute 'to_csv'
Sleeping for 0.43 seconds before trying the next player

 [3] Collecting stats for Rory Kerins at https://www.eliteprospects.com/player/504297/rory-kerins
Successfully scraped stats for Rory Kerins
Failed rate: 0.33
Sleep for 1.33 minutes to prevent getting blocked

 [4] Collecting stats for Oliver Moore at https://www.eliteprospects.com/player/603195/oliver-moore
Successfully scraped stats for Oliver Moore
Failed rate: 0.25
Sleep for 1.12 minutes to prevent getting blocked

 [5]

In [13]:
# Scrape players from 3900-len(nhl_players_metadata)
curr_batch_metadata = nhl_players_metadata[3900:]
get_players_stats_by_batch(curr_batch_metadata)


 [1] Collecting stats for Hunter Brzustewicz at https://www.eliteprospects.com/player/526088/hunter-brzustewicz
Successfully scraped stats for Hunter Brzustewicz
Failed rate: 0.00
Sleep for 0.47 minutes to prevent getting blocked

 [2] Collecting stats for Owen Sillinger at https://www.eliteprospects.com/player/199994/owen-sillinger
Successfully scraped stats for Owen Sillinger
Failed rate: 0.00
Sleep for 0.64 minutes to prevent getting blocked

 [3] Collecting stats for Nate Clurman at https://www.eliteprospects.com/player/353810/nate-clurman
Successfully scraped stats for Nate Clurman
Failed rate: 0.00
Sleep for 0.68 minutes to prevent getting blocked

 [4] Collecting stats for Donovan Sebrango at https://www.eliteprospects.com/player/397738/donovan-sebrango
Successfully scraped stats for Donovan Sebrango
Failed rate: 0.00
Sleep for 0.40 minutes to prevent getting blocked

 [5] Collecting stats for Kirill Kudryavtsev at https://www.eliteprospects.com/player/699365/kirill-kudryavtsev

### Fetch Players' Stats for Players Not Scraped

In [6]:
# Get unique players from nhl_players_stats.csv
if os.path.exists(output_path):
    # Read in the CSV file and set low_memory = False
    nhl_players_stats = pd.read_csv(output_path, low_memory=False)
    unique_players = set(nhl_players_stats['playername'].unique())
else:
    unique_players = set()

In [7]:
# Find players in nhl_players_metadata 0-2500 not in unique_players
subset = nhl_players_metadata[0:]
players_to_scrape = subset[~subset['playername'].isin(unique_players)].reset_index(drop=True)
players_to_scrape

,playername,fw_def,link
0,Victor Mancini,DEF,https://www.eliteprospects.com/player/290051/v...
1,Nolan Allan,DEF,https://www.eliteprospects.com/player/482413/n...
2,Colton Dach,FW,https://www.eliteprospects.com/player/413126/c...
3,Adam Wilsby,DEF,https://www.eliteprospects.com/player/295070/a...
4,Jimmy Snuggerud,FW,https://www.eliteprospects.com/player/623832/j...
5,Owen Pickering,DEF,https://www.eliteprospects.com/player/580641/o...
6,Oliver Kapanen,FW,https://www.eliteprospects.com/player/639018/o...
7,Alexander Nikishin,DEF,https://www.eliteprospects.com/player/514652/a...
8,Jacob Melanson,FW,https://www.eliteprospects.com/player/473633/j...
9,Dalibor Dvorský,FW,https://www.eliteprospects.com/player/527424/d...


In [8]:
curr_len = len(players_to_scrape)
# curr_len = 50
fail_count = 0

for i in range(curr_len):
    player_metadata = players_to_scrape.iloc[i]
    player_name = player_metadata['playername']
    player_url = player_metadata['link']
    print(f"\n [{i + 1}] Collecting stats for {player_name} at {player_url}")

    try:
        player_stats = ep.get_player_stats(player_metadata)

        # Write to CSV file
        if os.path.exists(output_path):
            player_stats.to_csv(output_path, mode='a', header=False, index=False, encoding='utf-8-sig')
        else:
            player_stats.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f'Successfully scraped stats for {player_name}')

        # Print Fail Rate
        print(f'Failed rate: {fail_count / (i + 1):.2f}')

        # Add random sleep to prevent getting blocked
        if i < curr_len - 1:
            sleep_time = random.uniform(10, 120)
            print(f"Sleep for {sleep_time / 60:.2f} minutes to prevent getting blocked")
            time.sleep(sleep_time)
    except Exception as e:
        print(f"Failed to get stats for {player_name}: {e}")

        fail_count += 1

        if i < curr_len - 1:
            # Sleep for 15-60 seconds before trying the next player
            sleep_time = random.uniform(15, 60)
            print(f"Sleeping for {sleep_time / 60:.2f} seconds before trying the next player")
            time.sleep(sleep_time)


 [1] Collecting stats for Victor Mancini at https://www.eliteprospects.com/player/290051/victor-mancini
Successfully scraped stats for Victor Mancini
Failed rate: 0.00
Sleep for 0.94 minutes to prevent getting blocked

 [2] Collecting stats for Nolan Allan at https://www.eliteprospects.com/player/482413/nolan-allan
Successfully scraped stats for Nolan Allan
Failed rate: 0.00
Sleep for 1.68 minutes to prevent getting blocked

 [3] Collecting stats for Colton Dach at https://www.eliteprospects.com/player/413126/colton-dach
Successfully scraped stats for Colton Dach
Failed rate: 0.00
Sleep for 0.74 minutes to prevent getting blocked

 [4] Collecting stats for Adam Wilsby at https://www.eliteprospects.com/player/295070/adam-wilsby
Successfully scraped stats for Adam Wilsby
Failed rate: 0.00
Sleep for 1.10 minutes to prevent getting blocked

 [5] Collecting stats for Jimmy Snuggerud at https://www.eliteprospects.com/player/623832/jimmy-snuggerud
Successfully scraped stats for Jimmy Snugger

In [9]:
# Check how many distinct players are in the output_path
nhl_players_stats = pd.read_csv(output_path)
unique_players = set(nhl_players_stats['playername'].unique())
print(f'Number of unique players: {len(unique_players)}')

Number of unique players: 3928


In [10]:
# Remove Duplicate rows in nhl_players_stats.csv
nhl_players_stats = pd.read_csv(output_path)
nhl_players_stats = nhl_players_stats.drop_duplicates().reset_index(drop=True)
nhl_players_stats.to_csv(output_path, index=False, encoding='utf-8-sig')